In [28]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
from datetime import datetime
from time import sleep
from random import randint
import pandas as pd
import requests, lxml, re, json

In [11]:
User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"
header={'User-Agent': User_Agent}

In [12]:
def page_next_hh(link):
    '''Функция возвращает текст кноки "дальше" при передачи в нее ссылки
    '''
    html = requests.get(link, headers={'User-Agent': User_Agent}).text
    parsed_html = bs(html,'lxml')
    page_next = parsed_html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'},).getText()
    return page_next

In [13]:
def page_next_sj(link):
    '''Функция возвращает текст кноки "дальше" при передачи в нее ссылки
    '''
    html = requests.get(link, headers={'User-Agent': User_Agent}).text
    parsed_html = bs(html,'lxml')
    page_next = parsed_html.find('a',{'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'},).getText()
    return page_next

In [14]:
def vacancies_hh(link):
    
    html = requests.get(link, headers={'User-Agent': User_Agent}).text
    parsed_html = bs(html,'lxml')
    vacancy_list = parsed_html.find_all('div',{'class':'vacancy-serp-item'},)

    for vacancy in vacancy_list:
        vac_data = {}

        info = vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})

        vacancy_name = info.getText()
        vacancy_link = info['href']
        site = 'hh.ru'

        zp = vacancy.find('div',{'class':'vacancy-serp-item__sidebar'}).findChild()
        if not zp:
            zp = None
            zp_min = None
            zp_max = None
            currency = None
        else:
            zp = zp.get_text()
            currency = (re.findall('USD',zp) or re.findall('руб.',zp) or re.findall('KZT',zp) or re.findall('EUR',zp) if zp else None)[0]
            if zp == None:
                zp_max = None
                zp_min = None
                currency = currency
            elif zp[:2] == 'от':
                zp_min = int(re.sub('\s+','',re.findall("\d+\s?\d+",zp)[0]))
                zp_max = None
                currency = currency
            elif zp[:2] == 'до':
                zp_min = None
                zp_max = int(re.sub('\s+','',re.findall("\d+\s?\d+",zp)[0]))
                currency = currency
            elif zp.find('-') != None:
                zp_min = int(re.sub('\s+','',re.findall("\d+\s?\d+",zp)[0]))
                zp_max = int(re.sub('\s+','',re.findall("\d+\s?\d+",zp)[1]))
                currency = currency
        
        vac_data['Сайт'] = site
        vac_data['Название вакансии'] = vacancy_name
        vac_data['От'] = zp_min
        vac_data['До'] = zp_max
        vac_data['Валюта'] = currency
        vac_data['Ссылка на вакансию'] = vacancy_link

        vacancies.append(vac_data)

In [15]:
def vacancies_sj(link):
    html = requests.get(link, headers=header).text
    parsed_html = bs(html,'lxml') 
    vacancy_blocks = parsed_html.find('div', {'style':"display:block"})    
    vacancy_block = vacancy_blocks.findChildren('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'}, recursive=False)
    
    for vacancy in vacancy_block:
        vac_data = {}
        
        pattern_name = re.compile('icMQ_ _1QIBo .*')
        name_module = vacancy.find('a', {'class': pattern_name})
        vacancy_name = name_module.text
        site = 'superjob.ru'
        vacancy_link =site + name_module['href']
        
        pattern_finance = re.compile('_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz')
        finance_module = vacancy.find('span', {'class': pattern_finance})
        finance = finance_module.text.replace('\xa0', ' ')
        if finance == 'По договорённости':
            zp_min = None
            zp_max = None
            currency = None
        else:        
            pattern_number = re.compile('[0-9]{1,3}\s{1}[0-9]{3}')
            pattren_valuta = re.compile('₽')
            pattren_ot = re.compile('от')
            pattren_do = re.compile('до')

            number = re.findall(pattern_number, finance)
            valuta = re.findall(pattren_valuta, finance)
            if valuta:
                valuta = valuta[0] 
            if len(number) == 2:
                zp_min = number[0]
                zp_max = number[1]
                currency = valuta
            elif len(number) == 1:
                ot = re.findall(pattren_ot, finance)
                do = re.findall(pattren_do, finance)
                if not do:
                    zp_min = number[0]
                    zp_max = None
                    currency = valuta
                elif not ot:
                    zp_min = None
                    zp_max = number[0]
                    currency = valuta
        
        vac_data['Сайт'] = site
        vac_data['Название вакансии'] = vacancy_name
        vac_data['От'] = zp_min
        vac_data['До'] = zp_max
        vac_data['Валюта'] = currency
        vac_data['Ссылка на вакансию'] = vacancy_link

        vacancies.append(vac_data)

In [16]:
def parse_hh(input_search):
    for page_number in range(0,1000):
        main_link = 'https://www.hh.ru'
        begin_link = '/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text='
        end_link = '&page='
        page = str(page_number)
        link = main_link + begin_link + input_search.replace(" ","+") + end_link + page

        vacancies_hh(link=link)
        try:
            page_next_hh(link=link)
        except AttributeError:
            break

In [17]:
def parse_sj(input_search):
    for page_number in range(0,1000):
        main_link = 'https://russia.superjob.ru'
        begin_link = '/vacancy/search/?keywords='
        end_link = '&page='
        page = str(page_number)
        link = main_link + begin_link+ input_search.replace(" ","%20") + end_link + page

        vacancies_sj(link=link)
        try:
            page_next_sj(link=link)
        except AttributeError:
            break

---

In [18]:
def total_parse(search):
    parse_hh(input_search=search)
    parse_sj(input_search=search)

## Итоговый парсинг

In [19]:
vacancies = []
# search = input('Введите название вакансии: ')
search = 'python'
total_parse(search)

In [20]:
# y = 'superjob.ru'
# x= '/vakansii/provizor-33574161.html'
# print(y+x)

---
# Итог

In [21]:
df = pd.DataFrame(vacancies)

In [22]:
df.loc[df['Валюта'] == 'руб.', 'Валюта'] = 'RUB'
df.loc[df['Валюта'] == '₽', 'Валюта'] = 'RUB'

In [23]:
print(f'размер = {df.shape}')
df.head()

размер = (2069, 6)


,Сайт,Название вакансии,От,До,Валюта,Ссылка на вакансию
0,hh.ru,Программист Python,95000,None,RUB,https://orenburg.hh.ru/vacancy/36293188?query=...
1,hh.ru,Fullstack Python Developer,1700,2000,USD,https://orenburg.hh.ru/vacancy/36222549?query=...
2,hh.ru,Программист-разработчик python (midle),90000,200000,RUB,https://orenburg.hh.ru/vacancy/35758988?query=...
3,hh.ru,Программист / Data Analyst,80000,None,RUB,https://orenburg.hh.ru/vacancy/36246190?query=...
4,hh.ru,Аналитик данных по продажам дилерской сети,None,None,None,https://orenburg.hh.ru/vacancy/36291421?query=...


In [24]:
df['Сайт'].value_counts()

hh.ru          2000
superjob.ru      69
Name: Сайт, dtype: int64

In [25]:
# print(datetime.today().strftime("%Y_%m_%d") )

In [26]:
df.to_excel(f'vacancies_{search}_{datetime.today().strftime("%Y_%m_%d")}.xlsx', 
            index = None, 
            header=True)

In [ ]:
# df.to_json(f'vacancies_{search}_{datetime.today().strftime("%Y_%m_%d")}.json',orient='index')

In [29]:
with open(f'vacancies_{search}_{datetime.today().strftime("%Y_%m_%d")}.json', "w",
          encoding="utf-8") as file:
    json.dump(vacancies, file)